In [2]:
import pandas as pd
from pathlib import Path
#from updates_join_annotations import open_df_from_tsv
#import updates_get_average_scores_per_label as scores


In [56]:
# Read in file and set data type for two columns
filepath = './label_level_df_sorted_cleaned.xlsx'
df = pd.read_excel(filepath, dtype={'tokens': list, 'annotators':str})
#df = pd.read_excel(filepath, dtype={'tokens': list, 'annotators':list})

#filepath = './label_level_df_sorted_cleaned.tsv'
#df = pd.read_csv(filepath, sep="\t", header=0, encoding="utf-8")



In [57]:
# Sort values and reset index to allow looping later
df.sort_values(by=['file_id', 'start_index', 'end_index', 'Label'], inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 1,Unnamed: 2,Label1,Label2,Label,file_id,start_index,end_index,tokens,annotators,annotator_count
0,12129,2503,48,54,view\,Patient,view\Patient,2503,48,54,"['Verteld', 'zelf', 'dat', 'hij', 'erg', 'prod...",{'katsburg'},1
1,0,2503,96,98,.B152: Stemming,NaN,.B152: Stemming,2503,96,98,"['Mw', 'was', 'emotioneel']",{'vanderpas'},1
2,10635,2503,96,98,STM 1,NaN,STM 1,2503,96,98,"['Mw', 'was', 'emotioneel']",{'vanderpas'},1
3,1,2503,97,98,.B152: Stemming,NaN,.B152: Stemming,2503,97,98,"['was', 'emotioneel']",{'opsomer'},1
4,2,2503,98,98,.B152: Stemming,NaN,.B152: Stemming,2503,98,98,['emotioneel'],"{'swartjes', 'avelli', 'vervaart', 'bos', 'mes...",6


In [58]:
#Found this on internet.....

import ast
from ast import literal_eval

def perfectEval(anonstring):
    try:
        ev = ast.literal_eval(anonstring)
        return ev
    except ValueError:
        corrected = "\'" + anonstring + "\'"
        ev = ast.literal_eval(corrected)
        return ev

In [62]:
def convertAnnotators1(annos):
    currentannos = annos
    if  type(annos)==str:
        currentannos =literal_eval(annos)
        #print('current was str')
    elif type(annos)==set:
        #<class 'pandas.core.series.Series'>                        
        currentannos = annos
        #print('current was set')
        
    elif type(annos)==list:
        currentannos = set(annos)
    else:
        corrected = "\'" + annos + "\'"
        #ev = ast.literal_eval(corrected)
        currentannos = annos
        #print('current was not str', )
    return(currentannos)

In [72]:
def convertAnnotators(annos):
    annolist =[]
    if type(annos) != str:
        for anno in annos:
            if anno.startswith('{'):
                anno = anno[2:-2]
            annolist.append(anno)
    else:
        anno = annos[2:-2]
        annolist.append(anno)
    newset = set(annolist)
    return(newset)

In [76]:
#file_id = 2345
#label = 'STM0'
label_dict = dict()

# Loop through file_ids (change later)
for file_id in set(df['file_id']):
#for file_id in [2503]:
    # Create filtered df
    df_filtered = df[df['file_id'] == file_id]
    # Loop through labels and create further filtered df
    for label in set(df_filtered['Label']):
                
        print(file_id, label)
        # df_label has the selection of the rows with the same label
        df_label = df_filtered[df_filtered['Label'] == label]
        df_label.reset_index(inplace=True, drop=True)
        
        try:
            # Set start and end index values
            start_index = df_label['start_index'][0]
            end_index = df_label['end_index'][0]
        except:
            continue
        
        
        #print('first annotation:', start_index, end_index)
        firstannotation = True
        counter = len(label_dict)+1
        #print(df_label.head(1)['tokens'])
        #print(df_label.head(1)['annotators'])
        label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': df_label.head(1)['tokens'],
                                           'annotators': convertAnnotators(df_label.head(1)['annotators'])}
        current_range = range(start_index, end_index+1)

        if df_label.shape[0] == 1:
            #print('There is only one annotation for this label so we are done.')
            # If only one tag of label in this file, we are done
            continue
 

        # There are multiple annotations so we need to check their spans and merge
        # label through rest of df
        for index, row in df_label[1:].iterrows():
            # Note: df which it is looping over contains only data from one file from one label

            start_temp = row['start_index']
            end_temp = row['end_index']
            #print('temp', start_temp, end_temp)


            # Since the df_label is sorted on start & end index, only a few cases need to be checked
            # If the temporary start vallue is in the range of the saved start value
            if start_temp in current_range:
                #print('included')
                #print('counter', counter)
                if end_temp not in current_range: # And the temp end value is not, then update the end value
                    #print('updated')
                    end_index = end_temp
                    ### we need to adapt the range
                    current_range = range(start_index, end_index+1)
                # If the temp end value is also in the range then do nothing and move on.
                ### we update the info for this annotation
                if counter in label_dict:
                    #print('counter in label_dict and adding annotators', start_index, end_index)
                    label_dict[counter]['end_index'] = end_index
                    
                    #print('label_dict[counter][annotators]', label_dict[counter]['annotators'])
                    #print(type(label_dict[counter]['annotators']))
                          
                    currentannos = convertAnnotators(label_dict[counter]['annotators'])
                    #print('currentannos', currentannos)
                    #print('type currentannos', type(currentannos))
                          
                    newannos = literal_eval(row['annotators'])
                    #print(type(newannos))
                    #print('new', newannos)
                    if type(currentannos)==set:
                        currentannos.update(newannos)
                    else:
                        currentannos = set(currentannos)
                    
                    currentannos.update(newannos)
                    label_dict[counter]['annotators'] = currentannos
                    print('Extended annotators:',label_dict[counter]['annotators'])
                else:
                    #print('counter not in label_dict', start_index, end_index)
                    label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}
                
            else:
                
                ### We have a new first annotation with start and range
                start_index = start_temp
                end_index = end_temp
                current_range = range(start_index, end_index+1)
                counter = len(label_dict)+1
                label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}

                #print('new first annotation:', start_index, end_index)
                #print('counter', counter)
                
            # For last entry
            if index == len(df_label)-1:
                print('last, stored')
                if counter in label_dict:
                    label_dict[counter]['end_index'] = end_index
                    
                    currentannos = convertAnnotators(label_dict[counter]['annotators'])
                    #print('currentannos', currentannos)
                    #print('type currentannos', type(currentannos))
                          
                    newannos = literal_eval(row['annotators'])
                    #print(type(currentannos))
                    #print('current', currentannos)
                    #print(type(newannos))
                    #print('new', newannos)
                    
                    currentannos.update(newannos)
                    label_dict[counter]['annotators'] = currentannos

                    print('Extended annotators:',label_dict[counter]['annotators'])
                   
                else: 
                    label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}
    


4100 FAC 1
Extended annotators: {'ske', 'vanderpas'}
Extended annotators: {'', 'nderp', 'swartjes'}
Extended annotators: {'', 'artj', 'e', 'avelli'}
last, stored
Extended annotators: {'', 'avelli', 'el'}
4100 FAC 2
Extended annotators: {'bos', 'tsbu'}
last, stored
Extended annotators: {'', 'bos'}
4100 type\Implicit
last, stored
Extended annotators: {'', 'swartjes'}
4100 INS 3
4100 .D450: Lopen en zich verplaatsen
Extended annotators: {'', 'swartjes', 'katsburg', 'meskers', 'avelli', 'bos'}
Extended annotators: {'', 'swartjes'}
last, stored
Extended annotators: {'', 'artj', 'swartjes'}
4100 FAC 0
4100 .B455: Inspanningstolerantie
Extended annotators: {'el', 'swartjes'}
last, stored
Extended annotators: {'', 'artj', 'swartjes'}
4100 view\Patient
last, stored
Extended annotators: {'', 'katsburg'}
4103 INS 2
last, stored
Extended annotators: {'', 'meskers'}
4103 type\Implicit
Extended annotators: {'', 'meskers'}
last, stored
Extended annotators: {'', 'meskers'}
4103 stm\reaction
Extended a

In [77]:
#### The next function could be differentiated per label
#- gemiddelde aantal gedeelde annotaties per label
#- hoeveel annotaties per student
#- hoe vaak is student outlier
#- confusion matrix
from collections import Counter

labels = []
annotators = []
annotation_total = 0
for key, value in label_dict.items():
    label = label_dict[key]['label']
    labels.append(label)
    key_annotators=label_dict[key]['annotators']
    for anno in key_annotators:
        annotators.append(anno)
    annotation_total += len(key_annotators)

print('Annotations per label', Counter(labels))
print('Annotations per annotator',Counter(annotators))
print('Total annotations:', len(label_dict.items()))
print('Total student annotations:', annotation_total)
print('Average agreement of annotations', annotation_total/(len(label_dict.items())))

Annotations per label Counter({'.B152: Stemming': 125, 'view\\Patient': 117, 'type\\Implicit': 113, 'STM 1': 108, 'stm\\reaction': 75, '.D450: Lopen en zich verplaatsen': 70, 'FAC 4': 61, '.B455: Inspanningstolerantie': 55, 'view\\Third party': 51, 'STM 2': 38, 'STM 0': 37, 'STM 3': 34, 'FAC 2': 26, 'INS 2': 25, 'FAC 3': 23, 'INS 1': 22, '.D840-859: Beroep en werk': 21, 'FAC 1': 19, 'FAC 5': 16, 'INS 3': 15, 'FAC 0': 13, 'INS 4': 7, 'STM 4': 6, 'BER 0': 5, '*': 4, 'INS 0': 4, 'INS 5': 3, 'BER 2': 3, 'BER 4': 2, 'BER 1': 2})
Annotations per annotator Counter({"'": 514, '': 468, 's': 308, 'katsburg': 269, 'e': 241, 'r': 233, 'a': 224, 'swartjes': 223, 'meskers': 219, '{': 185, '}': 185, 'avelli': 168, 't': 144, 'bos': 143, 'vervaart': 137, 'k': 124, 'ske': 99, 'b': 95, 'artj': 83, 'v': 81, 'u': 77, 'g': 77, 'tsbu': 75, 'rvaa': 74, 'el': 73, 'opsomer': 73, ',': 72, ' ': 72, 'm': 52, 'l': 50, 'w': 48, 'j': 48, 'vanderpas': 45, 'som': 41, 'nderp': 38, 'o': 28, 'i': 25, 'p': 23, 'n': 18, 'd'

In [78]:
df_soft = pd.DataFrame.from_dict(label_dict, orient='index')

In [79]:
df_soft.head()

,file_id,start_index,end_index,label,tokens,annotators
1,4100,515,524,FAC 1,"0 ['Kan', 'hierdoor', 'niet', 'zonder', 'st...","{, avelli, el}"
2,4100,517,520,FAC 2,"0 ['niet', 'zonder', 'steun'] Name: tokens,...","{, bos}"
3,4100,507,507,type\Implicit,"0 ['mobiliseren'] Name: tokens, dtype: object",{swartjes}
4,4100,556,558,type\Implicit,"['onvermogen', 'te', 'mobiliseren']","{, swartjes}"
5,4100,157,158,INS 3,"0 ['Patiente', 'fietste'] Name: tokens, dty...",{avelli}


In [80]:
df_soft.to_excel('softboundary_output_file.xlsx')